# Install Modules

In [1]:
!pip install -U transformers==3.0.0 --quiet
!python -m nltk.downloader punkt
!pip install pywsd
!pip install -U wn==0.0.22
!git clone https://github.com/ishita-2097/question_generation.git

     |████████████████████████████████| 754 kB 8.3 MB/s 
     |████████████████████████████████| 1.2 MB 46.4 MB/s 
     |████████████████████████████████| 3.0 MB 43.1 MB/s 
     |████████████████████████████████| 895 kB 53.9 MB/s 
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 26.8 MB 2.1 MB/s 
     |████████████████████████████████| 75 kB 230 kB/s 
  Created wheel for pywsd: filename=pywsd-1.2.4-py3-none-any.whl size=26940436 sha256=d2e845e3570d677fa2a95512bf594f91e08b9a179fc2e86c6f3450f40eb3edba
  Stored in directory: /root/.cache/pip/wheels/56/67/c0/6e6fa8456d1374b393328368316c3b33844cb4043bd225bc66
Successfully built pywsd
     |████████████████████████

In [2]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 7.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=46b81a3f97cf21587721fbb85988cc1b630ef277c72a26ba945418b52fa8888d
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [3]:
!pip install Flask-WTF
!pip install Flask-Bootstrap4

     |████████████████████████████████| 136 kB 8.3 MB/s 
     |████████████████████████████████| 1.7 MB 8.0 MB/s 
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3946 sha256=f47e8f62dec58b57615af43a46b9c0bc65c44604895896e230866aeb1a8895a8
  Stored in directory: /root/.cache/pip/wheels/64/34/11/053f47218984c9a31a00f911ed98dda036b867481dcc527a12
Successfully built visitor


In [4]:
%cd question_generation
from pipelines import pipeline as p_pipeline
from transformers import pipeline as t_pipeline
%cd /content

/content/question_generation
/content


In [6]:
import time
import nltk
from pywsd.similarity import max_similarity
import requests
import json
from wn import WordNet
from pywsd.lesk import adapted_lesk
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
nltk.download('popular')
nltk.download('punkt')
from nltk import tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data] 

In [7]:
from pyngrok import ngrok
from flask import Flask, render_template , request, send_file
import os
from flask_bootstrap import Bootstrap
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, TextAreaField
from wtforms.validators import DataRequired

In [8]:
nlp = p_pipeline("question-generation")
summarizer = t_pipeline("summarization")

Downloading:   0%|          | 0.00/627 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [9]:
def summ(input):
  summary = input.replace("\n", "")
  return summary

In [10]:
def get_wordsense(sent,word):
    word= word.lower()
    
    if len(word.split())>0:
        word = word.replace(" ","_")
    
    synsets = wn.synsets(word,'n')
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None

def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0: 
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors


def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = [] 
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term'] 

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)
                   
    return distractor_list

In [11]:
def filter_length_answers(json_result):
      result = []
      for qa_pair in json_result:
          lenght_of_answer = len(qa_pair["answer"]) 
          result.append(qa_pair)
      return result

In [12]:
def into_chunks(sent_tokenize):
  N=15
  chunk_list=[]
  msg=""
  for index, x in enumerate(sent_tokenize):
    msg+=(x+" ")
    if index % N ==N-1:
      chunk_list.append(msg)
      msg=""
  if msg:
    chunk_list.append(msg)
  return chunk_list

In [13]:
import sys

In [20]:
def func(input_text):

  sent_tokenize=tokenize.sent_tokenize(input_text)
  # print("I am here:",sent_tokenize)
  # sys.stdout.flush()

  input_text_mod=into_chunks(sent_tokenize)

  appended_summ=""
  # print(input_text_mod)
  # sys.stdout.flush()

  i=0
  for chunk_text in input_text_mod:
    word_count = len(chunk_text.split(' '))
    # print(i, word_count)
    appended_summ += summarizer(chunk_text, do_sample=False, max_length=word_count)[0]['summary_text']+" "
    i+=1

  # print("this is it:", appended_summ)
  summary = appended_summ.replace("\n", "")

  # input_text_mod=summarizer(input_text, do_sample=False, min_length=(int)(50*word_count/100))[0]['summary_text']
  # input_text_mod=summarizer(input_text, do_sample=False)[0]['summary_text']
  # summary=summ(input_text_mod)
  # print(summary)

  json_result = nlp(summary) 

  json_result = filter_length_answers(json_result)

  key_distractor_list = {}
  keyword_sentence_mapping = json_result

  SAlist = []

  for dict_QA in keyword_sentence_mapping:
    wordsense = get_wordsense(dict_QA['question'],dict_QA['answer'])
    if wordsense:
      distractors = get_distractors_wordnet(wordsense,dict_QA['answer'])
      if len(distractors) ==0:
        distractors = get_distractors_conceptnet(dict_QA['answer'])
      if len(distractors) != 0:
        key_distractor_list[dict_QA['answer']] = [dict_QA['question']]+distractors
      else:
        SAlist.append({'question':dict_QA['question'], 'ans':dict_QA['answer']})
    else:   
      distractors = get_distractors_conceptnet(dict_QA['answer'])
      if len(distractors) != 0:
        key_distractor_list[dict_QA['answer']] = [dict_QA['question']]+distractors
      else:
        SAlist.append({'question':dict_QA['question'], 'ans':dict_QA['answer']})

  QADlist = []

  for ans, value in key_distractor_list.items():
    d = dict()
    d['ans'] = ans
    d['question'] = value[0]
    d['distractor'] = value[1:]

    QADlist.append(d)

  return QADlist, SAlist

In [21]:
func("Information retrieval in computing and information science is the process of obtaining information system resources that are relevant to an information need from a collection of those resources. Searches can be based on fulltext or other contentbased indexing. Information retrieval is the science of searching for information in a document, searching for documents themselves, and also searching for the metadata that describes data, and for databases of texts, images or sounds. Automated information retrieval systems are used to reduce what has been called information overload. An IR system is a software system that provides access to books, journals and other documents stores and manages those documents. Web search engines are the most visible IR applications.")

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1500: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


([],
 [{'ans': 'information retrieval',
   'question': 'What is the process of obtaining information system resources that are relevant to an information need?'},
  {'ans': 'fulltext or other contentbased indexing',
   'question': 'What can search be based on?'},
  {'ans': 'automated information retrieval systems',
   'question': 'What is used to reduce what has been called information overload?'},
  {'ans': 'web search engines',
   'question': 'What are the most visible ir applications?'}])

In [22]:
func("Information retrieval in computing and information science is the process of obtaining information system resources that are relevant to an information need from a collection of those resources. Searches can be based on fulltext or other contentbased indexing. Information retrieval is the science of searching for information in a document, searching for documents themselves, and also searching for the metadata that describes data, and for databases of texts, images or sounds. Automated information retrieval systems are used to reduce what has been called information overload. An IR system is a software system that provides access to books, journals and other documents stores and manages those documents. Web search engines are the most visible IR applications.")

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1500: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


([],
 [{'ans': 'information retrieval',
   'question': 'What is the process of obtaining information system resources that are relevant to an information need?'},
  {'ans': 'fulltext or other contentbased indexing',
   'question': 'What can search be based on?'},
  {'ans': 'automated information retrieval systems',
   'question': 'What is used to reduce what has been called information overload?'},
  {'ans': 'web search engines',
   'question': 'What are the most visible ir applications?'}])

In [23]:
func("Information retrieval in computing and information science is the process of obtaining information system resources that are relevant to an information need from a collection of those resources. Searches can be based on fulltext or other contentbased indexing. Information retrieval is the science of searching for information in a document, searching for documents themselves, and also searching for the metadata that describes data, and for databases of texts, images or sounds. Automated information retrieval systems are used to reduce what has been called information overload. An IR system is a software system that provides access to books, journals and other documents stores and manages those documents. Web search engines are the most visible IR applications. Information retrieval in computing and information science is the process of obtaining information system resources that are relevant to an information need from a collection of those resources. Searches can be based on fulltext or other contentbased indexing. Information retrieval is the science of searching for information in a document, searching for documents themselves, and also searching for the metadata that describes data, and for databases of texts, images or sounds. Automated information retrieval systems are used to reduce what has been called information overload. An IR system is a software system that provides access to books, journals and other documents stores and manages those documents. Web search engines are the most visible IR applications.")

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1500: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


([],
 [{'ans': 'information retrieval',
   'question': 'What is the science of searching for information in a document?'},
  {'ans': 'automated information retrieval systems',
   'question': 'What are used to reduce what has been called information overload?'},
  {'ans': 'an ir system',
   'question': 'What is a software system that provides access to books, journals and other documents stores?'}])

In [27]:
func("Reading has many advantages. It gives us pleasure, increases our knowledge and sharpens our intellect. In the words of Francis Bacon, “Reading maketh a full man; conference a ready man and writing an exact man.” By reading, one becomes a complete man. One develops a comprehensive personality. One acquires knowledge and wisdom. Books widen our outlook on men and matters. By reading good books, we acquire noble habits.Reading is a source of great pleasure. It gives us a sense of satisfaction. It bestows peace of mind. Of course, only a person who has developed the habit of reading can reap all the benefits of reading. Therefore, everyone should develop a regular habit of reading. When we are in the grip of disappointment and despair, only reading of good books can give us peace of mind and a ray of hope for the future. Books inspire us to work hard to achieve success in life.Reading develops our latent qualities. Just as food is necessary for our body, reading is necessary for our mind. Reading is useful only if we read good books. Only good books can lead to a noble and successful career in life. Bad books spoil our character. It is said that if wealth is lost, nothing is lost, if health is lost, something is lost, but if character is lost, everything is lost. We should, therefore, not accept any bad book even for cursory reading.According to Bacon, some books are to be tasted, others to be swallowed and some few to be chewed and digested. A good book should, therefore, be chewed and digested so that it has a lasting influence on our life. For example, reading of the Gita leaves a lifelong influence on our mind. It teaches us the philosophy of life. It teaches us morality. It explains to us the Law of Karma, which means that we reap what we sow; our good actions alone will entitle us to a place in heaven. It exhorts us to do our duty and not bother about its result.The selection of books depends on one’s taste or aptitude. Young boys and girls usually like to read cheap romantic or sensational novels. Reading of these novels no doubt gives them a thrilling experience. But it does not help them in their studies. It does not improve their career. Reading of such books is, therefore, a sheer waste of time.Unfortunately, there is no one to guide the students as to what type of books they should read. It will be in the interest of students if they read only those books, magazines and periodicals which increase their knowledge of current, national and international affairs. They should also read biographies of great men like Mahatma Gandhi, Jawahar Lal Nehru, Subhash Chandra Bose, etc. They should also study the sacred books of various religions like the Vedas, Upanishads, the Gita, Quran, Bible, Granth Sahib, etc.Reading for pleasure is quite different from reading for a course or profession. When we read anything under compulsion, we do not enjoy it. But when we read for pleasure, we enjoy a lot. But, whatever be our motive, we should not read each and every book that comes in our way. We should be selective. We should separate the chaff from the grain. We should select the best books.The habit of reading should be developed from childhood itself so that when the children grow up, they are quite knowledgeable and can be successfiil in life. Books are no doubt our best companions. They never let us down.In the modern age, there is a great advancement in printing technology. Therefore, a large number of international and national newspapers are printed daily. Dozens of international and national magazines are also printed. Similarly magazines and periodicals which increase our general knowledge about science and current affairs are also published.These newspapers and magazines greatly enhance the knowledge of young men and women. Therefore, the student must read these newspapers and magazines. If he cannot buy these, he can go to some library and read the same. This will not only increase his knowledge but also develop a habit of reading them.'Abraham Lincoln' was born on February 12, 1809 in Kentucky (United States). His father's name was Thomas Lincoln and mother's name was Nancy Hanks Lincoln. He was born into a poor family, who did not have a good home to live in nor any means of getting education for children. Lincoln used to seek books from different places and read at night in the light of the stove fire and gained knowledge. He was self-educated and became a lawyer.Abraham Lincoln became the President of America. He was the sixteenth President of the United States. His tenure was from 1861 to 1865. He was a skilled politician as well as a book-lover, serious thinker and writer. He saved the country from being divided into two parts forever and also freed the country from the horribly inhuman slave system. Abraham Lincoln overcame America with its greatest crisis- the Civil War. Lincoln owes the end of slavery in America. He was assassinated on April 15, 1865. Lincoln is remembered as the martyr hero of the United States and he is consistently ranked as one of the greatest presidents in American history. Lincoln Memorial is one of the most visited monuments in the nation's capital.")

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1500: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size
Your min_length is set to 56, but you input_length is only 18. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)


([{'ans': 'waste of time',
   'distractor': ['Boondoggle',
    'Extravagance',
    'Squandering',
    'Waste Of Effort',
    'Waste Of Material',
    'Waste Of Money',
    'Waste Of Time'],
   'question': 'What is reading of cheap romantic or sensational novels?'},
  {'ans': 'kentucky',
   'distractor': ['Kansas',
    'New England',
    'Florida',
    'Montana',
    'Twin',
    'Alabama',
    'Yosemite',
    'Connecticut',
    'Mid-Atlantic states',
    'New Mexico'],
   'question': 'Where was abraham lincoln born?'},
  {'ans': 'lincoln memorial',
   'distractor': ['Inland Passage',
    'Lake Chelan',
    'scablands',
    'White House',
    'Mount Rainier National Park',
    'Columbia',
    'Bellingham',
    'Olympic National Park',
    'Ranier',
    'Mount Saint Helens'],
   'question': 'What is one of the most visited monuments in the United States?'}],
 [{'ans': 'great pleasure', 'question': 'What is reading a source of?'},
  {'ans': 'a person who has developed the habit of reading'

In [26]:
func("Rajiv Ratn Shah received his Ph.D. in computer science from the National University of Singapore, Singapore. Prior completing his Ph.D., he received his M.Tech. and MCA degrees in computer applications from the Delhi Technological University, Delhi and Jawaharlal Nehru University, Delhi, respectively. He has also received his BSc in Mathematics (Honors) from the Banaras Hindu University, Varanasi. Dr Shah is the recipient of several awards, including the prestigious European Research Consortium for Informatics and Mathematics (ERCIM) Fellowship and runner-up in the Grand Challenge competition of ACM International Conference on Multimedia. He is involved in organizing and reviewing of many top-tier international conferences and journals. After joining IIITD in Dec 2016, he has spent 1 year as a Research Fellow in Living Analytics Research Center (LARC) at the Singapore Management University, Singapore.")

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1500: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


([],
 [{'ans': 'national university of singapore, singapore',
   'question': 'Where did rajiv ratn shah receive his ph.d. in computer science?'},
  {'ans': 'rajiv ratn shah',
   'question': 'Who received his ph.d. in computer science from the national university of singapore?'},
  {'ans': 'european research consortium for informatics and mathematics (ercim) fellowship',
   'question': 'What award does rajiv ratn shah receive?'}])

In [28]:
class InputForm(FlaskForm):
    inputText = TextAreaField('Enter Quiz Material', validators=[DataRequired()],render_kw={'rows':'10'})
    submit = SubmitField('Generate Questions')

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
from google.colab.output import eval_js

In [31]:
# app = Flask(__name__,template_folder='/content/gdrive/MyDrive/IR_code/templates',static_folder='/content/gdrive/MyDrive/IR_code/static')
app = Flask(__name__,template_folder='/content/drive/MyDrive/Sem2/IR/project/UI/IR_project-main/templates',static_folder='/content/drive/MyDrive/Sem2/IR/project/UI/IR_project-main/static')
# ngrok.set_auth_token("27eM94qtit5orPFHdi2ZynwCk45_24A5V5MWUSx5o2ECzKiuh")
ngrok.set_auth_token("28LPt0m3AUQsPOSH2WFh6kzKhxa_2WbsruHArW1r9ZL8pDr4a")
port = 5000
public_url = ngrok.connect(port).public_url
print(eval_js("google.colab.kernel.proxyPort(5000)"))
app.config["BASE_URL"] = public_url
app.config['SECRET_KEY'] = 'C2HWGVoMGfNTBsrYQg8EcMrdTimkZfAb'
Bootstrap(app)

https://jb3mhsdm5q-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [32]:
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/quizgen',methods=["GET", "POST"])
def quizgen():
    
    form = InputForm()
    qa = []
    sa = []
    error = ""
    time_taken = -1
    if form.validate_on_submit():
        try:
          inputData = form.inputText.data
          start=time.time_ns()
          qa, sa = func(inputData)
          end=time.time_ns()
          time_taken=round((end-start)/1e9, 2)
          print(qa, sa)
          print("Time taken:::",time_taken)
          with open('filename.txt', 'w') as convert_file:
            convert_file.write(json.dumps(qa+sa))
        except Exception as e:
          qa=[]
          sa=[]
          error = "Error occurred: "+str(e)
          print(error)
        #return render_template('quizgen.html',form=form,result=qa)
    return render_template('quizgen.html',form=form,resultsQAD=qa, resultsSA=sa, error=error, time_taken=time_taken)

@app.route('/download')
def download():
	path = "filename.txt"
	return send_file(path, as_attachment=True)
 
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
app.run()

 * ngrok tunnel "http://4bcf-34-82-235-248.ngrok.io" -> "http://127.0.0.1:5000"
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Apr/2022 20:02:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2022 20:02:40] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2022 20:02:46] "GET /static/img/icons8-test-50.png HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2022 20:02:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2022 20:02:52] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Apr/2022 20:02:54] "GET /quizgen HTTP/1.1" 200 -
/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1500: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size
127.0.0.1 - - [26/Apr

[{'ans': 'psychology', 'question': 'What is the scientific study of the mind and behavior?', 'distractor': ['Agrobiology', 'Agrology', 'Agronomy', 'Architectonics', 'Cognitive Science', 'Cryptanalysis', 'Information Science', 'Linguistics', 'Mathematics', 'Metallurgy', 'Metrology', 'Natural History', 'Natural Science', 'Nutrition', 'Social Science', 'Strategics', 'Systematics', 'Thanatology']}, {'ans': 'two', 'question': 'How many dominant theoretical perspectives were there in the early days of psychology?', 'distractor': ['Binary Digit', 'Decimal Digit', 'Duodecimal Digit', 'Eight', 'Five', 'Four', 'Hexadecimal Digit', 'Nine', 'Octal Digit', 'One', 'Seven', 'Significant Digit', 'Six', 'Three', 'Zero']}, {'ans': 'structuralism', 'question': 'What was the name given to the approach pioneered by wilhelm wundt?', 'distractor': ['Atomism', 'Blastogenesis', 'Economic Theory', 'Field Theory', 'Holism', 'Preformation', 'Reductionism', 'Scientific Theory']}] [{'question': 'What is the origin 

127.0.0.1 - - [26/Apr/2022 20:04:44] "POST /quizgen HTTP/1.1" 200 -


[] [{'question': 'Where did rajiv ratn shah receive his ph.d. in computer science?', 'ans': 'national university of singapore, singapore'}, {'question': 'Who received his ph.d. in computer science from the national university of singapore?', 'ans': 'rajiv ratn shah'}, {'question': 'What award does rajiv ratn shah receive?', 'ans': 'european research consortium for informatics and mathematics (ercim) fellowship'}]
Time taken::: 20.35


...

[{'ans': 'half', 'question': 'How many respondents said they were cooking more than before the pandemic?', 'distractor': ['One-billionth', 'One-eighth', 'One-fifth', 'One-fourth', 'One-half', 'One-hundred-millionth', 'One-hundred-thousandth', 'One-hundredth', 'One-millionth', 'One-ninth', 'One-quadrillionth', 'One-quintillionth', 'One-seventh', 'One-sixteenth', 'One-sixth', 'One-sixtieth', 'One-sixty-fourth', 'One-ten-thousandth', 'One-tenth', 'One-third', 'One-thirty-second', 'One-thousandth', 'One-trillionth', 'One-twelfth', 'Three-fourths', 'Two-thirds']}] [{'question': 'When have sales of processed and “comfort” foods taken off?', 'ans': 'early march'}, {'question': 'What is adequate for the purchase of healthy food to cook?', 'ans': 'food safety net'}, {'question': 'What is changing our relationship with food well beyond the kitchen?', 'ans': 'the pandemic'}, {'question': 'When was home gardening on rise?', 'ans': 'great recession of 2008'}, {'question': 'Home gardening is on rise

127.0.0.1 - - [26/Apr/2022 21:01:52] "POST /quizgen HTTP/1.1" 200 -


[] [{'question': 'What is reading a source of?', 'ans': 'great pleasure'}, {'question': 'Who can reap all the benefits of reading?', 'ans': 'a person who has developed the habit of reading'}, {'question': 'How does one become a person who has developed the habit of reading?', 'ans': 'complete man'}, {'question': 'What does one acquire by reading good books?', 'ans': 'knowledge and wisdom'}, {'question': 'What does one acquire by reading good books?', 'ans': 'noble habits'}, {'question': 'What is the name of a book that is to be tasted, swallowed and digested?', 'ans': 'bacon'}, {'question': 'What can a good book lead to?', 'ans': 'noble and successful career in life'}, {'question': 'What type of books do young boys and girls like to read?', 'ans': 'cheap romantic or sensational novels'}, {'question': 'What is reading of cheap romantic or sensational novels?', 'ans': 'sheer waste of time'}, {'question': 'What should students read to increase their knowledge of current, national and inte

127.0.0.1 - - [26/Apr/2022 21:07:16] "POST /quizgen HTTP/1.1" 200 -


[{'ans': 'infrastructure', 'question': "The rapid urbanization will put a strain on governments' ability to provide basic services such as sewerage, water, electricity and what else?", 'distractor': ['Computer Architecture', 'Cytoarchitecture', 'Framework']}] [{'question': 'What has put humanity and the world at peril?', 'ans': 'rapid global population increase'}, {'question': 'How many people live in poverty?', 'ans': '6 billion'}, {'question': "What is the world's population growing in the developing world?", 'ans': 'fastest'}, {'question': "How much of the world's population is expected to hit in developing countries?", 'ans': '9 billion'}, {'question': 'What is the result of the growing population in the developing world?', 'ans': 'population growth is declining in the developed world'}, {'question': 'How many people cannot sustain the population of the developing world?', 'ans': '10 billion'}, {'question': 'How long is the population heading in the aging direction?', 'ans': '50 ye

In [ ]:
"ONE DOLLAR AND EIGHTY-SEVEN CENTS. That was all. And sixty cents of it was in pennies. Pennies saved one and two at a time by bulldozing the grocer and the vegetable man and the butcher until one's cheek burned with the silent imputation of parsimony that such close dealing implied. Three times Della counted it. One dollar and eighty-seven cents. And the next day would be Christmas. There was clearly nothing left to do but flop down on the shabby little couch and howl. So Della did it. Which instigates the moral reflection that life is made up of sobs, sniffles, and smiles, with sniffles predominating. While the mistress of the home is gradually subsiding from the first stage to the second, take a look at the home. A furnished flat at $8 per week. It did not exactly beggar description, but it certainly had that word on the look-out for the mendicancy squad. In the vestibule below was a letter-box into which no letter would go, and an electric button from which no mortal finger could coax a ring. Also appertaining thereunto was a card bearing the name 'Mr. James Dillingham Young.' The 'Dillingham' had been flung to the breeze during a former period of prosperity when its possessor was being paid $30 per week. Now, when the income was shrunk to $20, the letters of 'Dillingham' looked blurred, as though they were thinking seriously of contracting to a modest and unassuming D. But whenever Mr. James Dillingham Young came home and reached his flat above he was called 'Jim' and greatly hugged by Mrs. James Dillingham Young, already introduced to you as Della. Which is all very good. Delia finished her cry and attended to her cheeks with the powder rag. She stood by the window and looked out dully at a grey cat walking a grey fence in a grey backyard. To-morrow would be Christmas Day, and she had only $1.87 with which to buy Jim a present. She had been saving every penny she could formonths, with this result. Twenty dollars a week doesn't go far. Expenses had been greater than she had calculated. They always are. Only $1.87 to buy a present for Jim. Her Jim. Many a happy hour she had spent planning for something nice for him. Something fine and rare and sterling - something just a little bit near to being worthy of the honour of being owned by Jim. There was a pier-glass between the windows of the room. Perhaps you have seen a pier-glass in an $8 flat. A very thin and very agile person may, by observing his reflection in a rapid sequence of longitudinal strips, obtain a fairly accurate conception of his looks. Della, being slender, had mastered the art. Suddenly she whirled from the window and stood before the glass. Her eyes were shining brilliantly, but her face had lost its colour within twenty seconds. Rapidly she pulled down her hair and let it fall to its full length. Now, there were two possessions of the James Dillingham Youngs in which they both took a mighty pride. One was Jim's gold watch that had been his father's and his grandfather's. The other was Della's hair. Had the Queen of Sheba lived in the flat across the airshaft, Della would have let her hair hang out the window some day to dry just to depreciate Her Majesty's jewels and gifts. Had King Solomon been the janitor, with all his treasures piled up in the basement, Jim would have pulled out his watch every time he passed, just to see him pluck at his beard from envy. So now Della's beautiful hair fell about her, rippling and shining like a cascade of brown waters. It reached below her knee and made itself almost a garment for her. And then she did it up again nervously and quickly. Once she faltered for a minute and stood still while a tear or two splashed on the worn red carpet. On went her old brown jacket; on went her old brown hat. With a whirl of skirts and with the brilliant sparkle still in her eyes, she fluttered out of the door and down the stairs to the street. Where she stopped the sign read: 'Mme. Sofronie. Hair Goods of All Kinds.' One flight up Della ran, and collected herself, panting. Madame, large, too white, chilly, hardly looked the 'Sofronie.' 'Will you buy my hair?' asked Della. 'I buy hair,' said Madame. 'Take yer hat off and let's have a sight at the looks of it.' Down rippled the brown cascade. 'Twenty dollars,' said Madame, lifting the mass with a practised hand."

In [ ]:
"Information retrieval in computing and information science is the process of obtaining information system resources that are relevant to an information need from a collection of those resources. Searches can be based on full-text or other content-based indexing. Information retrieval is the science of searching for information in a document, searching for documents themselves, and also searching for the metadata that describes data, and for databases of texts, images or sounds. Automated information retrieval systems are used to reduce what has been called information overload. An IR system is a software system that provides access to books, journals and other documents; stores and manages those documents. Web search engines are the most visible IR applications."

In [ ]:
"Psychology is the scientific study of the mind and behavior, according to the American Psychological Association. Psychology is a multifaceted discipline and includes many sub-fields of study such areas as human development, sports, health, clinical, social behavior and cognitive processes. Psychology is really a very new science, with most advances happening over the past 150 years or so.  However, its origins can be traced back to ancient Greece, 400 – 500 years BC. In the early days of psychology there were two dominant theoretical perspectives regarding how the brain worked, structuralism and functionalism. Structuralism was the name given to the approach pioneered by Wilhelm Wundt (1832-1920), which focused on breaking down mental processes intro the most basic components.The term originated from Edward Titchener, an American psychologist who had been trained by Wundt. Wundt was important because he separated psychology from philosophy by analyzing the workings of the mind in a more structured way, with the emphasis being on objective measurement and control.Structuralism relied on trained introspection, a research method whereby subjects related what was going on in their minds while performing a certain task.However, introspection proved to be an unreliable method because there was too much individual variation in the experiences and reports of research subjects. Despite the failure of introspection Wundt is an important figure in the history of psychology as he opened the first laboratory dedicated to psychology in 1879, and its opening is usually thought of as the beginning of modern experimental psychology. James argued that the mind is constantly changing and it is pointless to look for the structure of conscious experience. Rather, he proposed the focus should be on how and why an organism does something, i.e. the functions or purpose of the brain.James suggested that psychologists should look for the underlying cause of behavior and the mental processes involved. This emphasis on the causes and consequences of behavior has influenced contemporary psychology."

**Sample texts:**  

* Rajiv Ratn Shah received his Ph.D. in computer science from the National University of Singapore, Singapore. Prior completing his Ph.D., he received his M.Tech. and MCA degrees in computer applications from the Delhi Technological University, Delhi and Jawaharlal Nehru University, Delhi, respectively. He has also received his BSc in Mathematics (Honors) from the Banaras Hindu University, Varanasi. Dr Shah is the recipient of several awards, including the prestigious European Research Consortium for Informatics and Mathematics (ERCIM) Fellowship and runner-up in the Grand Challenge competition of ACM International Conference on Multimedia. He is involved in organizing and reviewing of many top-tier international conferences and journals. After joining IIITD in Dec 2016, he has spent 1 year as a Research Fellow in Living Analytics Research Center (LARC) at the Singapore Management University, Singapore.  



*   Reading has many advantages. It gives us pleasure, increases our knowledge and sharpens our intellect. In the words of Francis Bacon, “Reading maketh a full man; conference a ready man and writing an exact man.” By reading, one becomes a complete man. One develops a comprehensive personality. One acquires knowledge and wisdom. Books widen our outlook on men and matters. By reading good books, we acquire noble habits.Reading is a source of great pleasure. It gives us a sense of satisfaction. It bestows peace of mind. Of course, only a person who has developed the habit of reading can reap all the benefits of reading. Therefore, everyone should develop a regular habit of reading. When we are in the grip of disappointment and despair, only reading of good books can give us peace of mind and a ray of hope for the future. Books inspire us to work hard to achieve success in life.Reading develops our latent qualities. Just as food is necessary for our body, reading is necessary for our mind. Reading is useful only if we read good books. Only good books can lead to a noble and successful career in life. Bad books spoil our character. It is said that if wealth is lost, nothing is lost, if health is lost, something is lost, but if character is lost, everything is lost. We should, therefore, not accept any bad book even for cursory reading.According to Bacon, some books are to be tasted, others to be swallowed and some few to be chewed and digested. A good book should, therefore, be chewed and digested so that it has a lasting influence on our life. For example, reading of the Gita leaves a lifelong influence on our mind. It teaches us the philosophy of life. It teaches us morality. It explains to us the Law of Karma, which means that we reap what we sow; our good actions alone will entitle us to a place in heaven. It exhorts us to do our duty and not bother about its result.The selection of books depends on one’s taste or aptitude. Young boys and girls usually like to read cheap romantic or sensational novels. Reading of these novels no doubt gives them a thrilling experience. But it does not help them in their studies. It does not improve their career. Reading of such books is, therefore, a sheer waste of time.Unfortunately, there is no one to guide the students as to what type of books they should read. It will be in the interest of students if they read only those books, magazines and periodicals which increase their knowledge of current, national and international affairs. They should also read biographies of great men like Mahatma Gandhi, Jawahar Lal Nehru, Subhash Chandra Bose, etc. They should also study the sacred books of various religions like the Vedas, Upanishads, the Gita, Quran, Bible, Granth Sahib, etc.Reading for pleasure is quite different from reading for a course or profession. When we read anything under compulsion, we do not enjoy it. But when we read for pleasure, we enjoy a lot. But, whatever be our motive, we should not read each and every book that comes in our way. We should be selective. We should separate the chaff from the grain. We should select the best books.The habit of reading should be developed from childhood itself so that when the children grow up, they are quite knowledgeable and can be successfiil in life. Books are no doubt our best companions. They never let us down.In the modern age, there is a great advancement in printing technology. Therefore, a large number of international and national newspapers are printed daily. Dozens of international and national magazines are also printed. Similarly magazines and periodicals which increase our general knowledge about science and current affairs are also published.These newspapers and magazines greatly enhance the knowledge of young men and women. Therefore, the student must read these newspapers and magazines. If he cannot buy these, he can go to some library and read the same. This will not only increase his knowledge but also develop a habit of reading them.'Abraham Lincoln' was born on February 12, 1809 in Kentucky (United States). His father's name was Thomas Lincoln and mother's name was Nancy Hanks Lincoln. He was born into a poor family, who did not have a good home to live in nor any means of getting education for children. Lincoln used to seek books from different places and read at night in the light of the stove fire and gained knowledge. He was self-educated and became a lawyer.Abraham Lincoln became the President of America. He was the sixteenth President of the United States. His tenure was from 1861 to 1865. He was a skilled politician as well as a book-lover, serious thinker and writer. He saved the country from being divided into two parts forever and also freed the country from the horribly inhuman slave system. Abraham Lincoln overcame America with its greatest crisis- the Civil War. Lincoln owes the end of slavery in America. He was assassinated on April 15, 1865. Lincoln is remembered as the martyr hero of the United States and he is consistently ranked as one of the greatest presidents in American history. Lincoln Memorial is one of the most visited monuments in the nation's capital.

*   Psychology is the scientific study of the mind and behavior, according to the American Psychological Association. Psychology is a multifaceted discipline and includes many sub-fields of study such areas as human development, sports, health, clinical, social behavior and cognitive processes. Psychology is really a very new science, with most advances happening over the past 150 years or so.  However, its origins can be traced back to ancient Greece, 400 – 500 years BC. In the early days of psychology there were two dominant theoretical perspectives regarding how the brain worked, structuralism and functionalism. Structuralism was the name given to the approach pioneered by Wilhelm Wundt (1832-1920), which focused on breaking down mental processes intro the most basic components.The term originated from Edward Titchener, an American psychologist who had been trained by Wundt. Wundt was important because he separated psychology from philosophy by analyzing the workings of the mind in a more structured way, with the emphasis being on objective measurement and control.Structuralism relied on trained introspection, a research method whereby subjects related what was going on in their minds while performing a certain task.However, introspection proved to be an unreliable method because there was too much individual variation in the experiences and reports of research subjects. Despite the failure of introspection Wundt is an important figure in the history of psychology as he opened the first laboratory dedicated to psychology in 1879, and its opening is usually thought of as the beginning of modern experimental psychology. James argued that the mind is constantly changing and it is pointless to look for the structure of conscious experience. Rather, he proposed the focus should be on how and why an organism does something, i.e. the functions or purpose of the brain.James suggested that psychologists should look for the underlying cause of behavior and the mental processes involved. This emphasis on the causes and consequences of behavior has influenced contemporary psychology.

* Information retrieval in computing and information science is the process of obtaining information system resources that are relevant to an information need from a collection of those resources. Searches can be based on fulltext or other contentbased indexing. Information retrieval is the science of searching for information in a document, searching for documents themselves, and also searching for the metadata that describes data, and for databases of texts, images or sounds. Automated information retrieval systems are used to reduce what has been called information overload. An IR system is a software system that provides access to books, journals and other documents stores and manages those documents. Web search engines are the most visible IR applications.

